In [3]:
from pymongo import MongoClient
import pandas as pd
import re
import csv
import json
import random
import time
import calendar
from datetime import datetime
# Importamos cliente de Mongodb para Python
from pymongo import MongoClient

# Conectamos a Mongodb. En nuestro caso, estamos levantando el servidor en local (host es, por tanto, localhost)
# y el puerto será el de por defecto, 27017
client = MongoClient("localhost:27017")
db = client.admin
print("Client admin creado.")

# Imprimimos el resultado del comando serverStatus
serverStatusResult = db.command("serverStatus")
print("Server status:")
print(serverStatusResult)

Client admin creado.
Server status:
{'host': 'CMEMBRILLATP', 'version': '5.0.3', 'process': 'C:\\Program Files\\MongoDB\\Server\\5.0\\bin\\mongod.exe', 'pid': 5164, 'uptime': 249466.0, 'uptimeMillis': 249466508, 'uptimeEstimate': 249466, 'localTime': datetime.datetime(2022, 6, 10, 11, 45, 46, 816000), 'asserts': {'regular': 0, 'warning': 0, 'msg': 0, 'user': 283, 'tripwire': 0, 'rollovers': 0}, 'catalogStats': {'collections': 22, 'capped': 0, 'timeseries': 0, 'views': 0, 'internalCollections': 4, 'internalViews': 0}, 'connections': {'current': 5, 'available': 999995, 'totalCreated': 120, 'active': 2, 'threaded': 5, 'exhaustIsMaster': 0, 'exhaustHello': 0, 'awaitingTopologyChanges': 1}, 'electionMetrics': {'stepUpCmd': {'called': 0, 'successful': 0}, 'priorityTakeover': {'called': 0, 'successful': 0}, 'catchUpTakeover': {'called': 0, 'successful': 0}, 'electionTimeout': {'called': 0, 'successful': 0}, 'freezeTimeout': {'called': 0, 'successful': 0}, 'numStepDownsCausedByHigherTerm': 0, 

In [4]:
# Conectamos a Mongodb. En nuestro caso, estamos levantando el servidor en local (host es, por tanto, localhost)
# y el puerto será el de por defecto, 27017
client = MongoClient("localhost:27017")
db = client.admin
print("Client admin creado.")

# Imprimimos el resultado del comando serverStatus
serverStatusResult = db.command("serverStatus")
print("Server status:")
print(serverStatusResult)

Client admin creado.
Server status:
{'host': 'CMEMBRILLATP', 'version': '5.0.3', 'process': 'C:\\Program Files\\MongoDB\\Server\\5.0\\bin\\mongod.exe', 'pid': 5164, 'uptime': 249466.0, 'uptimeMillis': 249466573, 'uptimeEstimate': 249466, 'localTime': datetime.datetime(2022, 6, 10, 11, 45, 46, 881000), 'asserts': {'regular': 0, 'warning': 0, 'msg': 0, 'user': 285, 'tripwire': 0, 'rollovers': 0}, 'catalogStats': {'collections': 22, 'capped': 0, 'timeseries': 0, 'views': 0, 'internalCollections': 4, 'internalViews': 0}, 'connections': {'current': 7, 'available': 999993, 'totalCreated': 123, 'active': 3, 'threaded': 7, 'exhaustIsMaster': 0, 'exhaustHello': 0, 'awaitingTopologyChanges': 2}, 'electionMetrics': {'stepUpCmd': {'called': 0, 'successful': 0}, 'priorityTakeover': {'called': 0, 'successful': 0}, 'catchUpTakeover': {'called': 0, 'successful': 0}, 'electionTimeout': {'called': 0, 'successful': 0}, 'freezeTimeout': {'called': 0, 'successful': 0}, 'numStepDownsCausedByHigherTerm': 0, 

In [5]:
# Definición bases de datos
db = client["db"]
collectionMovies = "movies"
collectionUsers = "users"
collectionViews = "views"
collectionLikes = "likes"
collectionFollows = "follows"
collectionMoviesList = "movieslist"

#Definición de variables

#films
idString = "id"
titleString = "title"
ratingString = "rating"
yearString = "year"
usersratingString = "users_rating"    
votesString = "votes"
metascoreString = "metascore"
countriesString = "countries"
languagesString = "languages"
actorsString = "actors"
genresString = "genres"
descriptionString = "description"
directorsString = "directors"
runtimeString = "runtime"
directorString = "director"
codirectorString = "codirector"
genreString = "genre"
cogenreString = "cogenre"
mainactorString = "mainactor"
secactorString = "secactor"
voseString = "vose"
seclanguageString = "seclanguage"
countryString = "country"
seccountryString = "seccountry"
emptyString = ""

#users

nombreString = "nombre"
apellidosString = "apellidos"
correoString = "correo"
nacimientoString = "nacimiento"
postalString = "postal"

#views
completedString= "completed"
scoreString = "score"
userString = "user"
titleString = "title"
yearString = "year"
timestampString = "timestamp"
newVotesString = "votesFilmin"
newRatingString = "ratingFilmin"
completedRuntimeString = "completedRuntime"
likeString = "like"
viewingtimeString = "viewingtime"

#rating

votesString = "votes"
usersratingString = "users_rating"   

#follow

userString = "user"
followString = "follow"


In [6]:
def getSampleMovie():
    # Usamos el comando aggregate para lanzar un aggregation pipeline de Mongodb
    resultadoAgg = db[collectionMovies].aggregate([
            { "$sample":{ "size": 1 } }
        ])

    # Recorremos el resultado
    for movie in resultadoAgg:
        #print(movie)
        return movie


In [7]:
def getSampleUser():
    # Usamos el comando aggregate para lanzar un aggregation pipeline de Mongodb
    resultadoAgg = db[collectionUsers].aggregate([
            { "$sample":{ "size": 1 } }
        ])

    # Recorremos el resultado
    for user in resultadoAgg:
        #print(user)
        return user

In [8]:
def getCurrentTimestamp():
    gmt = time.gmtime()
    ts = calendar.timegm(gmt)
    return ts

In [9]:
def getRandomInt(min, max):
    return random.randint(min, max)

In [10]:
def cleanRuntime(runtime):
    if runtime != None and isinstance(runtime, str):
        runtime = int(runtime.replace(" min", ''))

    return runtime

In [9]:
def generateView(user):   
    if user is None:
        userObject = getSampleUser()
        user = userObject.get(correoString)
        #print(user)
    else:
        userObject = user
        user = userObject.get(correoString)
        #print(str(user))
    
    movieObject = getSampleMovie()
    #print(str(movieObject))

    # Recogemos los datos de la película
    movieObject = db[collectionMovies].find_one({titleString: movieObject.get(titleString)})
        
    view = {}
    
    view[userString] = userObject.get(correoString)
    view[titleString] = movieObject.get(titleString)
    view[yearString] = movieObject.get(yearString)
    view[timestampString] = getCurrentTimestamp()
    
    runtime = movieObject.get(runtimeString)
    viewingtime = getRandomInt(int(cleanRuntime(runtime) * 0.15), int(cleanRuntime(runtime)))
    
    if viewingtime > cleanRuntime(runtime)* 0.8 :
        completed= True
    else:
        completed= False 

    
    view[completedString] = completed
    view [viewingtimeString] =viewingtime
    
    if completed == True :
        view[scoreString] = getRandomInt(0, 10)         
    
    return view


In [10]:
def generateLike(view):   
#Genero el like a partir de la vista
    like = {}
    like[userString] = view[userString]
    like[titleString] = view[titleString]
    like[yearString] = view[yearString]
    
    return like

In [11]:
def updateRatingFilm(view):
    
    title =view[titleString]
    year= view[yearString]
    
    movieObject = db[collectionMovies].find_one({titleString: title, yearString: year})
    
    newRating = 0
    newVotes = 0
    newScore = int(view[scoreString])
    totalVotes = int(movieObject[votesString])
    totalRating = float(movieObject[usersratingString])
                
    newVotes = totalVotes + 1
    newRating = ((totalRating * totalVotes) + newScore)/newVotes

    print("Old votes: " + str(totalVotes))    
    print("New votes: " + str(newVotes))
    print("Old users rating: " + str(totalRating))
    print("New users rating: " + str(newRating))
    output = db[collectionMovies].update_one({titleString: title}, {'$set': {usersratingString: newRating, votesString: newVotes }}) 

    # Pintamos los documentos modificados
    print('Documentos modificados: ' + str(output.modified_count))
    

In [12]:
def insertView (view):
#inserto en vista y si le ha gustado inserto en peliculas que gustan

    db[collectionViews].insert_one(view)

    score = view.get(scoreString)

    if score :
        updateRatingFilm(view)
        if score >= 7 : 
            like = {}
            like = generateLike(view)
            db[collectionLikes].insert_one(like)

In [23]:
def generateMassiveViews(views):   
    for i in range(views):
        view = {}
        view = generateView(None)
        insertView(view)
        
generateMassiveViews(10)

Old votes: 161679
New votes: 161680
Old users rating: 7.2
New users rating: 7.200017318159327
Documentos modificados: 1
Old votes: 79139
New votes: 79140
Old users rating: 7.8
New users rating: 7.799977255496588
Documentos modificados: 1
Old votes: 18911
New votes: 18912
Old users rating: 6.6
New users rating: 6.599862521150592
Documentos modificados: 1
Old votes: 5924
New votes: 5925
Old users rating: 5.7
New users rating: 5.699375527426161
Documentos modificados: 1


IndentationError: unexpected indent (<ipython-input-14-988ea43f3d17>, line 14)

In [15]:
def arreglaVotos():

    movies = db[collectionMovies].find({})

    for m in movies:
        print(m)

        votes = m[votesString]
        title = m[titleString]
        
        votes = int(votes.replace(',', ''))

        output = db[collectionMovies].update_one({titleString: title}, {'$set': {votesString: votes}}) 

        # Pintamos los documentos modificados
        print('Documentos modificados: ' + str(output.modified_count))       

arreglaVotos()

{'_id': ObjectId('62920b14f70f528bb91f841c'), 'title': 'A Rainy Day in New York', 'rating': 'PG-13', 'year': '2019', 'users_rating': '6.6', 'votes': 21903, 'metascore': '44', 'description': 'A young couple arrives in New York for a weekend where they are met with bad weather and a series of adventures.', 'runtime': '92 min', 'codirector': None, 'director': 'Woody Allen', 'cogenre': 'Romance', 'genre': 'Comedy', 'mainactor': 'Timothée Chalamet', 'secactor': 'Elle Fanning', 'seclanguage': None, 'vose': 'English', 'country': 'USA', 'seccountry': None}


AttributeError: 'int' object has no attribute 'replace'

In [13]:
def insertFollow (follow):
#inserto el follow
    follows = db[collectionFollows].count_documents({userString: follow[userString], followString: follow[followString]})
    ##para comprobar si existe ya el follow antes de insertar
    if follows > 0:
        print("El follow de " + follow[userString] +" a " +follow[followString]+" ya existe")
    else:
        db[collectionFollows].insert_one(follow)

In [20]:
def generateFollow(user, follow): 
    print("Estoy aqui")
    continua = True
    if user is None:
        user = getSampleUser()
        usermail = user.get(correoString)
        print("user "+usermail)
    else:
        userObject = user
        usermail = userObject.get(correoString)
        print("user "+str(usermail))
    if follow is None:
        while continua :
            follow = getSampleUser()
            followmail = follow.get(correoString)
            print("follow "+followmail+" y usermail "+usermail)
            if usermail != followmail :
                continua = False
        print("follow "+followmail)
    else:
        followmail = follow.get(correoString)
        if usermail != followmail :
            continua = True
            while follow is None or continua:
                follow = getSampleUser()
                followmail = follow.get(correoString)
                print("follow "+followmail)
                if usermail != followmail :
                    continua = False
        print("follow "+followmail)

    #Genero el follow
    follow = {}
    follow[userString] = usermail
    follow[followString] = followmail
    
    return follow

In [43]:
def generateMassiveFollows(follows):   
    for i in range(follows):
        follow = {}
        follow = generateFollow(None,None)
        insertFollow(follow)
        
generateMassiveFollows(10)

Estoy aqui
user rodrigo@gmail.com
follow pablo@gmail.com y usermail rodrigo@gmail.com
follow pablo@gmail.com
El follo de rodrigo@gmail.com a pablo@gmail.com ya existe
Estoy aqui
user raul@gmail.com
follow pablo@gmail.com y usermail raul@gmail.com
follow pablo@gmail.com
Estoy aqui
user benito@gmail.com
follow rodrigo@gmail.com y usermail benito@gmail.com
follow rodrigo@gmail.com
El follo de benito@gmail.com a rodrigo@gmail.com ya existe
Estoy aqui
user rodrigo@gmail.com
follow hilario@gmail.com y usermail rodrigo@gmail.com
follow hilario@gmail.com
El follo de rodrigo@gmail.com a hilario@gmail.com ya existe
Estoy aqui
user raul@gmail.com
follow pablo@gmail.com y usermail raul@gmail.com
follow pablo@gmail.com
El follo de raul@gmail.com a pablo@gmail.com ya existe
Estoy aqui
user rodrigo@gmail.com
follow pablo@gmail.com y usermail rodrigo@gmail.com
follow pablo@gmail.com
El follo de rodrigo@gmail.com a pablo@gmail.com ya existe
Estoy aqui
user benito@gmail.com
follow rodrigo@gmail.com y us

In [55]:
def unfollow(user, follow):
    
    usermail = user[correoString]
    followmail = follow[correoString]
    
    resultado = db[collectionFollows].delete_one({userString: usermail, followString: followmail}) 
    deletedDocs = resultado.deleted_count
    print(str(deletedDocs))

    print("Delete done.")

In [56]:
def unfollowForDelete (user):

    usermail = user[correoString]
    resultado = db[collectionFollows].delete_many( {userString: { "$eq": usermail } } )
    deletedDocs = resultado.deleted_count
    print("User follows deleted: "+str(deletedDocs))
    
    resultado = db[collectionFollows].delete_many( {followString: { "$eq": usermail } } )
    deletedDocs = resultado.deleted_count
    print("User follows deleted: "+str(deletedDocs))
    print("Delete done.")

In [64]:
def generateUnfollow(user, follow):   
    continua = True
    if user is None:
        user = getSampleUser()
        usermail = user.get(correoString)
        print("user "+usermail)
    else:
        userObject = user
        usermail = userObject.get(correoString)
        print("user "+str(usermail))
    if follow is None:
        while continua :
            follow = getSampleUser()
            followmail = follow.get(correoString)
            print("follow "+followmail+" y usermail "+usermail)
            if usermail != followmail :
                continua = False
        print("follow "+followmail)
    else:
        followmail = follow.get(correoString)
        if usermail != followmail :
            continua = True
            while follow is None or continua:
                follow = getSampleUser()
                followmail = follow.get(correoString)
                print("follow "+followmail)
                if usermail != followmail :
                    continua = False
        print("follow "+followmail) 
    
            
    follows = db[collectionFollows].count_documents({userString: usermail, followString: followmail})
    ##para comprobar si existe ya el follow antes de insertar
    print("follows: "+ str(follows))
    if follows > 0:
        unfollow(user, follow)
    else:
        print("El unfollow de " + usermail +" a " +followmail+" no existe")
        
        
generateUnfollow(None, None)

user pablo@gmail.com
follow pablo@gmail.com y usermail pablo@gmail.com
follow rodrigo@gmail.com y usermail pablo@gmail.com
follow rodrigo@gmail.com
follows: 0
El unfollow de pablo@gmail.com a rodrigo@gmail.com no existe


In [62]:
def generateUnfollowForDelete(user):   
    if user is None:
        user = getSampleUser()
        usermail = user.get(correoString)
        print("user "+usermail)
    else:
        userObject = user
        usermail = userObject.get(correoString)
        #print("user "+str(usermail))
    
    unfollowForDelete(user)
        
generateUnfollowForDelete(None)

user pablo@gmail.com
User follows deleted: 2
User follows deleted: 2
Delete done.


In [11]:
def insertMovieList (movielist):
    #inserto el movielist
    existe = db[collectionMoviesList].count_documents({userString: movielist[userString], titleString: movielist[titleString], yearString: movielist[yearString]})
    ##para comprobar si existe ya el follow antes de insertar
    if existe > 0:
        print("La pelicla " + movielist[titleString] +" ya está en la lista de " + movielist[userString])
    else:
        db[collectionMoviesList].insert_one(movielist)

In [16]:
def generateMovieList(user, movie): 
    
    if user is None:
        user = getSampleUser()
        usermail = user.get(correoString)
        print("user "+ usermail)
    else:
        userObject = user
        usermail = userObject.get(correoString)
        print("user "+str(usermail))
        
    if movie is None:
        movie = getSampleMovie()
        movietitle = movie.get(titleString)
        print("movie "+ movietitle)
    else:
        movietitle = movie.get(titleString)
        print("movie "+movietitle)

    #Genero el movielist
    movielist = {}
    movielist[userString] = user[correoString]
    movielist[titleString] = movie[titleString]
    movielist[yearString] = movie[yearString]
    
    return movielist

In [17]:
def generateMassiveMovieList(moviesonlist):   
    for i in range(moviesonlist):
        movielist = {}
        movielist = generateMovieList(None,None)
        insertMovieList(movielist)
        
generateMassiveMovieList(10)

user rodrigo@gmail.com
movie Take Care
user pablo@gmail.com
movie Navy Seals
user benito@gmail.com
movie Sunlight Jr.
user pablo@gmail.com
movie She Hate Me
user hilario@gmail.com
movie The Way Way Back
user raul@gmail.com
movie No Retreat, No Surrender
user hilario@gmail.com
movie Back in the Day
user pablo@gmail.com
movie Crank: High Voltage
user raul@gmail.com
movie Beautiful
user raul@gmail.com
movie Saw III


In [ ]:
def deleteMovieList(movielist):
    
    usermail = movielist[correoString]
    title = movielist[titleString]
    year = movielist[yearString]
    
    moviesonlist = db[collectionMoviesList].count_documents({userString: usermail, titleString: title, yearString: year})
    ##para comprobar si existe ya el follow antes de insertar
    print("follows: "+ str(follows))
    if moviesonlist > 0:
        resultado = db[collectionMoviesList].delete_one({userString: usermail, titleString: title, yearString: year}) 
        deletedDocs = resultado.deleted_count
        print(str(deletedDocs))
        print("Delete done.")
    else:
        print("El unfollow de " + usermail +" a " +followmail+" no existe")


    

In [ ]:
def deleteMovieListForDeleteUser (user):

    usermail = user[correoString]
    resultado = db[collectionMoviesList].delete_many( {userString: { "$eq": usermail } } )
    deletedDocs = resultado.deleted_count
    print("User moviesList deleted: "+str(deletedDocs))

In [ ]:
def generateDeleteMovieList(user, movie):   

    if user is None:
        user = getSampleUser()
        usermail = user.get(correoString)
        print("user "+ usermail)
    else:
        userObject = user
        usermail = userObject.get(correoString)
        print("user "+str(usermail))
        
    if movie is None:
        movie = getSampleMovie()
        movietitle = movie.get(titleString)
        print("movie "+ movietitle)
    else:
        movietitle = movie.get(titleString)
        print("movie "+movietitle)

    #Genero el movielist
    movielist = {}
    movielist[userString] = user[correoString]
    movielist[titleString] = movie[titleString]
    movielist[yearString] = movie[yearString]
    
return movielist
        
generateUnfollow(None, None)